<a href="https://colab.research.google.com/github/imnawar/AnnArabic/blob/master/Stacked-mnist/DCGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras.datasets.mnist import load_data
import matplotlib.pyplot as plt
import numpy as np 

(x_train, y_train), (x_test, y_test) = load_data()

def create_sn_batch(batch): 
  b_size = batch.shape[0]
  batch = np.reshape(batch, [b_size, 3, 28, 28])
  batch1 = batch[:int(b_size/2), :, :, :]
  batch2 = batch[int(b_size/2):, :, :, :]

  batch1 = np.reshape(batch1, [batch1.shape[0]*batch1.shape[1], 28, 28])
  batch2 = np.reshape(batch2, [batch2.shape[0]*batch2.shape[1], 28, 28])

  return batch1, batch2

def stacked_mnist_batch_data(data, labels, batch_size):
    nb_data = np.shape(data)[0]
    ridx_ch1 = np.random.randint(nb_data, size=batch_size);
    ridx_ch2 = np.random.randint(nb_data, size=batch_size);
    ridx_ch3 = np.random.randint(nb_data, size=batch_size);
    mnist_ch1 = data[ridx_ch1, :]
    mnist_ch2 = data[ridx_ch2, :]
    mnist_ch3 = data[ridx_ch3, :]
    mnist_im1 = np.reshape(mnist_ch1, [batch_size, 28, 28, 1])
    mnist_im2 = np.reshape(mnist_ch2, [batch_size, 28, 28, 1])
    mnist_im3 = np.reshape(mnist_ch3, [batch_size, 28, 28, 1])
    label_ch1 = labels[ridx_ch1]
    label_ch2 = labels[ridx_ch2]
    label_ch3 = labels[ridx_ch3]
    label_img1 = np.reshape(label_ch1, [batch_size, 1])
    label_img2 = np.reshape(label_ch2, [batch_size, 1])
    label_img3 = np.reshape(label_ch3, [batch_size, 1])
    batch = np.concatenate([mnist_im1, mnist_im2, mnist_im3], axis=3)
    batch_labels = np.concatenate([label_img1, label_img2, label_img3], axis=-1)
    batch = np.reshape(batch, [batch_size, 28, 28, 3])
    batch_labels = np.reshape(batch_labels, [batch_size, 3])
    # batch = [mnist_ch1, mnist_ch2, mnist_ch3]
    return batch, batch_labels

def create_sn_batch_from(batch): 
  ch1, ch2, ch3 = batch[:, :, :, 0], batch[:, :, :, 1], batch[:, :, :, 2]
  b_size = ch1.shape[0]
  # batch = np.reshape(batch, [b_size, 3, 28, 28])
  batch1_ch1 = ch1[:int(b_size/2), :, :]
  batch2_ch1 = ch1[int(b_size/2):, :, :]

  batch1_ch2 = ch2[:int(b_size/2), :, :]
  batch2_ch2 = ch2[int(b_size/2):, :, :]

  batch1_ch3 = ch3[:int(b_size/2), :, :]
  batch2_ch3 = ch3[int(b_size/2):, :, :]

  # batch1 = np.reshape(batch1, [batch1.shape[0]*batch1.shape[1], 28, 28])
  # batch2 = np.reshape(batch2, [batch2.shape[0]*batch2.shape[1], 28, 28])
  batch1 = [batch1_ch1, batch1_ch2, batch1_ch3]
  batch2 = [batch2_ch1, batch2_ch2, batch2_ch3]
  return batch1, batch2

11501568/11490434 [==============================] - 0s 0us/step


In [ ]:
# example of a dcgan on cifar10
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy import vstack
from numpy.random import randn
from numpy.random import randint
from keras.datasets.mnist import load_data
# from tf.keras.optimizers import Adam tf.keras.optimizers.Adam
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout
from matplotlib import pyplot
import tensorflow as tf
import matplotlib.pyplot as plt


def define_discriminator(in_shape=(28,28,3)):
	model = tf.keras.models.Sequential()
	model.add(tf.keras.layers.Conv2D(128, (5, 5),padding='same', input_shape=in_shape))
	model.add(tf.keras.layers.Activation('tanh'))
	model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
	model.add(tf.keras.layers.Flatten())
	model.add(tf.keras.layers.Dense(512))
	model.add(tf.keras.layers.Activation('tanh'))
	model.add(tf.keras.layers.Dense(1))
	model.add(tf.keras.layers.Activation('sigmoid'))
	# compile model
	opt = tf.keras.optimizers.Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
	return model

# define the standalone generator model
def define_generator(latent_dim):
	model = tf.keras.models.Sequential()
	model.add(tf.keras.layers.Dense((128*7*7), input_shape=(latent_dim,)))
	model.add(tf.keras.layers.BatchNormalization())
	model.add(tf.keras.layers.Activation('tanh'))
	model.add(tf.keras.layers.Reshape((7, 7, 128)))
	model.add(tf.keras.layers.UpSampling2D(size=(2, 2)))
	model.add(tf.keras.layers.Conv2D(64, (5, 5), padding='same'))
	model.add(tf.keras.layers.Activation('tanh'))
	model.add(tf.keras.layers.UpSampling2D(size=(2, 2)))
	model.add(tf.keras.layers.Conv2D(3, (5, 5), padding='same'))
	model.add(tf.keras.layers.Activation('tanh'))
	return model

# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model):
	# make weights in the discriminator not trainable
	d_model.trainable = False
	# connect them
	model = Sequential()
	# add generator
	model.add(g_model)
	# add the discriminator
	model.add(d_model)
	# compile model
	opt = tf.keras.optimizers.Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt)
	return model

# load and prepare cifar10 training images
def load_real_samples():
	# load cifar10 dataset
	# (trainX, _), (_, _) = load_data()
	trainX = x_train
	# convert from unsigned ints to floats
	X = trainX.astype('float32')
	# scale from [0,255] to [-1,1]
	X = X / 127.5 - 1.
	return X

# select real samples
def generate_real_samples(dataset, n_samples):
	# choose random instances
	# ix = randint(0, dataset.shape[0], n_samples)
	# retrieve selected images
	# X = dataset[ix]
	# generate 'real' class labels (1)
	X, _ = stacked_mnist_batch_data(dataset, y_train, n_samples)
	y = ones((n_samples, 1))
	return X, y

# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
	# generate points in the latent space
	x_input = randn(latent_dim * n_samples)
	# reshape into a batch of inputs for the network
	x_input = x_input.reshape(n_samples, latent_dim)
	return x_input

# use the generator to generate n fake examples, with class labels
def generate_fake_samples(g_model, latent_dim, n_samples):
	# generate points in latent space
	x_input = generate_latent_points(latent_dim, n_samples)
	# predict outputs
	X = g_model.predict(x_input)
	# create 'fake' class labels (0)
	y = zeros((n_samples, 1))
	return X, y

# # create and save a plot of generated images
def save_plot(examples, epoch, n=10):
	# scale from [-1,1] to [0,1]
	examples = (examples + 1) / 2.0
	# plot images
	for i in range(n * n):
		# define subplot
		pyplot.subplot(n, n, 1 + i)
		# turn off axis
		pyplot.axis('off')
		# plot raw pixel data
		pyplot.imshow(examples[i])
	# save plot to file
	filename = 'generated_plot_e%03d.png' % (epoch+1)
	pyplot.savefig(filename)
	pyplot.close()


# evaluate the discriminator, plot generated images, save generator model
def summarize_performance(epoch, g_model, d_model, dataset, latent_dim, n_samples=150):
	# prepare real samples
	X_real, y_real = generate_real_samples(dataset, n_samples)
	# evaluate discriminator on real examples
	_, acc_real = d_model.evaluate(X_real, y_real, verbose=0)
	# prepare fake examples
	x_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_samples)
	# evaluate discriminator on fake examples
	_, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)
	# summarize discriminator performance
	print('>Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real*100, acc_fake*100))
	# save plot
	save_plot(x_fake, epoch)
	# save the generator model tile file
	filename = 'generator_model_%03d.h5' % (epoch+1)
	g_model.save(filename)

# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=200, n_batch=128):
	bat_per_epo = int(dataset.shape[0] / n_batch)
	half_batch = int(n_batch / 2)
	# manually enumerate epochs
	for i in range(n_epochs):
		# enumerate batches over the training set
		for j in range(bat_per_epo):
			# get randomly selected 'real' samples
			X_real, y_real = generate_real_samples(dataset, half_batch)
			# update discriminator model weights
			d_loss1, _ = d_model.train_on_batch(X_real, y_real)
			# generate 'fake' examples
			X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
			# update discriminator model weights
			d_loss2, _ = d_model.train_on_batch(X_fake, y_fake)
			# prepare points in latent space as input for the generator
			X_gan = generate_latent_points(latent_dim, n_batch)
			# create inverted labels for the fake samples
			y_gan = ones((n_batch, 1))
			# update the generator via the discriminator's error
			g_loss = gan_model.train_on_batch(X_gan, y_gan)
			# summarize loss on this batch
			# print('>%d, %d/%d, d1=%.3f, d2=%.3f g=%.3f' %
			# 	(i+1, j+1, bat_per_epo, d_loss1, d_loss2, g_loss))
		# evaluate the model performance, sometimes
		# summarize_performance(i, g_model, d_model, dataset, latent_dim)
		# cal_fid(g_model)
		# cal_is(g_model)
		# if (i+1) % 10 == 0:
		# 	summarize_performance(i, g_model, d_model, dataset, latent_dim)
		# 	cal_fid(g_model)
		# 	cal_is(g_model)
			if (j+1) % bat_per_epo == 0:
				print('>%d, %d/%d, d1=%.3f, d2=%.3f g=%.3f' %
				(i+1, j+1, bat_per_epo, d_loss1, d_loss2, g_loss))
		if (i+1) % 10 == 0:
			summarize_performance(i, g_model, d_model, dataset, latent_dim)
			cal_fid(g_model)
			cal_is(g_model)



def cal_fid(generator):
	# Testing 
	import math
	import tensorflow as tf
	import time
	from numpy import cov
	from numpy import trace
	from numpy import iscomplexobj
	from numpy.random import random
	from scipy.linalg import sqrtm
	from numpy import expand_dims
	from keras.models import load_model
	from numpy.random import randn
	from keras.models import model_from_json
	from keras.datasets import mnist
	import numpy as np
	import random
	from keras.applications.inception_v3 import preprocess_input
	from skimage.transform import resize
	from keras.applications.inception_v3 import InceptionV3

	model = InceptionV3(include_top=False, pooling='avg', input_shape=(299,299,3))
	def scale_images(images, new_shape):
		images_list = list()
		for image in images:
			# resize with nearest neighbor interpolation
			new_image = resize(image, new_shape, 0)
			# store
			images_list.append(new_image)
		return np.asarray(images_list)
	
	# calculate frechet inception distance
	def calculate_fid(model, images1, images2):
		# calculate activations
		act1 = model.predict(images1)
		act2 = model.predict(images2)
		# calculate mean and covariance statistics
		mu1, sigma1 = act1.mean(axis=0), cov(act1, rowvar=False)
		mu2, sigma2 = act2.mean(axis=0), cov(act2, rowvar=False)
		# calculate sum squared difference between means
		ssdiff = np.sum((mu1 - mu2)**2.0)
		# calculate sqrt of product between cov
		covmean = sqrtm(sigma1.dot(sigma2))
		# check and correct imaginary numbers from sqrt
		if iscomplexobj(covmean):
			covmean = covmean.real
		# calculate score
		fid = ssdiff + trace(sigma1 + sigma2 - 2.0 * covmean)
		return fid

	for i in range(5):
		imgs_num = 500
		idx = np.random.randint(0, dataset.shape[0], imgs_num)
		# define two collections of 500 images
		reals = dataset[idx]
		# reals = reals.reshape((imgs_num,32,32,3))
		noise = np.random.normal(0, 1, (imgs_num, 100))
		# Generate a batch of new images
		fakes = generator.predict(noise)
		gen_imgs = fakes
		fakes = fakes.reshape((imgs_num,28,28,3))
		# resize images
		reals = scale_images(reals, (299,299,3))
		fakes = scale_images(fakes, (299,299,3))
		reals = preprocess_input(reals)
		fakes = preprocess_input(fakes)
		# fid between images1 and images2
		fid = calculate_fid(model, reals, fakes)
		print('FID: %.3f' % fid)


def cal_is(generator):
	# calculate inception score for cifar-10 in Keras
	from math import floor
	from numpy import ones
	from numpy import expand_dims
	from numpy import log
	from numpy import mean
	from numpy import std
	from numpy import exp
	from numpy.random import shuffle
	from keras.applications.inception_v3 import InceptionV3
	from keras.applications.inception_v3 import preprocess_input
	from keras.datasets import mnist
	from skimage.transform import resize
	from numpy import asarray

	# scale an array of images to a new size
	def scale_images(images, new_shape):
		images_list = list()
		for image in images:
			# resize with nearest neighbor interpolation
			new_image = resize(image, new_shape, 0)
			# store
			images_list.append(new_image)
		return asarray(images_list)

	# assumes images have any shape and pixels in [0,255]
	def calculate_inception_score(images, n_split=10, eps=1E-16):
		# load inception v3 model
		model = InceptionV3()
		# enumerate splits of images/predictions
		scores = list()
		n_part = floor(images.shape[0] / n_split)
		for i in range(n_split):
			# retrieve images
			ix_start, ix_end = i * n_part, (i+1) * n_part
			subset = images[ix_start:ix_end]
			# convert from uint8 to float32
			subset = subset.astype('float32')
			# scale images to the required size
			subset = scale_images(subset, (299,299,3))
			# pre-process images, scale to [-1,1]
			# subset = preprocess_input(subset)
			# predict p(y|x)
			p_yx = model.predict(subset)
			# calculate p(y)
			p_y = expand_dims(p_yx.mean(axis=0), 0)
			# calculate KL divergence using log probabilities
			kl_d = p_yx * (log(p_yx + eps) - log(p_y + eps))
			# sum over classes
			sum_kl_d = kl_d.sum(axis=1)
			# average over images
			avg_kl_d = mean(sum_kl_d)
			# undo the log
			is_score = exp(avg_kl_d)
			# store
			scores.append(is_score)
		# average across images
		is_avg, is_std = mean(scores), std(scores)
		return is_avg, is_std

	# load cifar10 images
	# noise = np.random.normal(0, 1, (1000, 100))
	noise = generate_latent_points(100, 1000)
	# Generate a batch of new images
	images = generator.predict(noise)
	shuffle(images)
	print('loaded', images.shape)
	# calculate inception score
	is_avg, is_std = calculate_inception_score(images)
	print('score', is_avg, is_std)

# def load_gen(g_model):
# 	generator = g_model
# 	generator.load_weights("/content/generator_model_001.h5")
# 	print("Loaded model from disk")
# 	return generator
# size of the latent space
latent_dim = 100
# create the discriminator
d_model = define_discriminator()
# create the generator
g_model = define_generator(latent_dim)
# create the gan
gan_model = define_gan(g_model, d_model)
# load image data
dataset = load_real_samples()
# train model
train(g_model, d_model, gan_model, dataset, latent_dim)
# genrator = load_gen(g_model)
# cal_fid(genrator)
# cal_is(genrator)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


>1, 468/468, d1=0.537, d2=0.542 g=1.722
>2, 468/468, d1=0.437, d2=0.508 g=1.814
>3, 468/468, d1=0.349, d2=0.326 g=1.838
>4, 468/468, d1=0.240, d2=0.296 g=2.367
>5, 468/468, d1=0.136, d2=0.167 g=3.060
>6, 468/468, d1=0.144, d2=0.106 g=3.631
>7, 468/468, d1=0.095, d2=0.070 g=4.097
>8, 468/468, d1=0.148, d2=0.177 g=3.834
>9, 468/468, d1=0.254, d2=0.197 g=3.497
>10, 468/468, d1=0.189, d2=0.155 g=3.737
>Accuracy real: 86%, fake: 100%
FID: 50.508
FID: 49.841
FID: 52.140
FID: 51.169
FID: 52.537
loaded (1000, 28, 28, 3)
96124928/96112376 [==============================] - 1s 0us/step
score 1.6471552 0.08900775
>11, 468/468, d1=0.263, d2=0.268 g=2.435
>12, 468/468, d1=0.218, d2=0.272 g=3.121
>13, 468/468, d1=0.260, d2=0.161 g=3.065
>14, 468/468, d1=0.224, d2=0.185 g=2.947


In [6]:
x_fake1, y_fake1 = generate_fake_samples(g_model, latent_dim, 150)
save_plot(x_fake1, 8)

In [5]:
# create and save a plot of generated images
def save_plot(examples, epoch, n=10):
	# scale from [-1,1] to [0,1]
	examples = (examples + 1) / 2.0
	# plot images
	for i in range(n * n):
		# define subplot
		pyplot.subplot(n, n, 1 + i)
		# turn off axis
		pyplot.axis('off')
		# plot raw pixel data
		pyplot.imshow(examples[i])
	# save plot to file
	filename = 'generated_plot_e%03d.png' % (epoch+1)
	pyplot.savefig(filename)
	pyplot.close()

# No need to the following cells: 

In [ ]:
y = 1 
def yy(y):
  y=2

yy(y)

y

1

In [ ]:

generator = g_model
generator.load_weights("/content/generator_model_200.h5")
print("Loaded model from disk")

Loaded model from disk


In [ ]:
import math
import tensorflow as tf
import time
from numpy import cov
from numpy import trace
from numpy import iscomplexobj
from numpy.random import random
from scipy.linalg import sqrtm
from numpy import expand_dims
from keras.models import load_model
from numpy.random import randn
from keras.models import model_from_json
from keras.datasets import mnist
import numpy as np
import random
from keras.applications.inception_v3 import preprocess_input
from skimage.transform import resize
from keras.applications.inception_v3 import InceptionV3

# (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
# x_train = x_train / 127.5 - 1.
# x_test = x_test / 127.5 - 1.
# print(x_train.shape)

# json_file = open('/content/output/generator.json', 'r')
# loaded_model_json = json_file.read()
# json_file.close()
# generator = model_from_json(loaded_model_json)
# generator.load_weights("/content/output/generator.h5")
# print("Loaded model from disk")

def scale_images(images, new_shape):
	images_list = list()
	for image in images:
		# resize with nearest neighbor interpolation
		new_image = resize(image, new_shape, 0)
		# store
		images_list.append(new_image)
	return np.asarray(images_list)
 
 # calculate frechet inception distance
def calculate_fid(model, images1, images2):
	# calculate activations
	act1 = model.predict(images1)
	act2 = model.predict(images2)
	# calculate mean and covariance statistics
	mu1, sigma1 = act1.mean(axis=0), cov(act1, rowvar=False)
	mu2, sigma2 = act2.mean(axis=0), cov(act2, rowvar=False)
	# calculate sum squared difference between means
	ssdiff = np.sum((mu1 - mu2)**2.0)
	# calculate sqrt of product between cov
	covmean = sqrtm(sigma1.dot(sigma2))
	# check and correct imaginary numbers from sqrt
	if iscomplexobj(covmean):
		covmean = covmean.real
	# calculate score
	fid = ssdiff + trace(sigma1 + sigma2 - 2.0 * covmean)
	return fid

model = InceptionV3(include_top=False, pooling='avg', input_shape=(299,299,3))

In [ ]:
for i in range(12):
    imgs_num = 500
    idx = np.random.randint(0, dataset.shape[0], imgs_num)
    # define two collections of 500 images
    reals = dataset[idx]
    # reals = reals.reshape((imgs_num,32,32,3))
    noise = np.random.normal(0, 1, (imgs_num, 100))
    # Generate a batch of new images
    fakes = g_model.predict(noise)
    gen_imgs = fakes
    fakes = fakes.reshape((imgs_num,32,32,3))
    # resize images
    reals = scale_images(reals, (299,299,3))
    fakes = scale_images(fakes, (299,299,3))
    reals = preprocess_input(reals)
    fakes = preprocess_input(fakes)
    # fid between images1 and images2
    fid = calculate_fid(model, reals, fakes)
    print('FID: %.3f' % fid)

FID: 0.973
FID: 0.831
FID: 1.136
FID: 0.952
FID: 1.471
FID: 1.123
FID: 0.799
FID: 0.979
FID: 1.045
FID: 1.247
FID: 1.321
FID: 1.021


In [ ]:
# calculate inception score for cifar-10 in Keras
from math import floor
from numpy import ones
from numpy import expand_dims
from numpy import log
from numpy import mean
from numpy import std
from numpy import exp
from numpy.random import shuffle
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras.datasets import mnist
from skimage.transform import resize
from numpy import asarray

# (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
# x_train = x_train / 127.5 - 1.
# x_test = x_test / 127.5 - 1.
# print(x_train.shape)


# scale an array of images to a new size
def scale_images(images, new_shape):
	images_list = list()
	for image in images:
		# resize with nearest neighbor interpolation
		new_image = resize(image, new_shape, 0)
		# store
		images_list.append(new_image)
	return asarray(images_list)

# assumes images have any shape and pixels in [0,255]
def calculate_inception_score(images, n_split=10, eps=1E-16):
	# load inception v3 model
	model = InceptionV3()
	# enumerate splits of images/predictions
	scores = list()
	n_part = floor(images.shape[0] / n_split)
	for i in range(n_split):
		# retrieve images
		ix_start, ix_end = i * n_part, (i+1) * n_part
		subset = images[ix_start:ix_end]
		# convert from uint8 to float32
		subset = subset.astype('float32')
		# scale images to the required size
		subset = scale_images(subset, (299,299,3))
		# pre-process images, scale to [-1,1]
		# subset = preprocess_input(subset)
		# predict p(y|x)
		p_yx = model.predict(subset)
		# calculate p(y)
		p_y = expand_dims(p_yx.mean(axis=0), 0)
		# calculate KL divergence using log probabilities
		kl_d = p_yx * (log(p_yx + eps) - log(p_y + eps))
		# sum over classes
		sum_kl_d = kl_d.sum(axis=1)
		# average over images
		avg_kl_d = mean(sum_kl_d)
		# undo the log
		is_score = exp(avg_kl_d)
		# store
		scores.append(is_score)
	# average across images
	is_avg, is_std = mean(scores), std(scores)
	return is_avg, is_std

# load cifar10 images
# (images, _), (_, _) = mnist.load_data()
# shuffle images
images_reals = dataset[:1000]
shuffle(images_reals)
print('loaded', images_reals.shape)
# calculate inception score
is_avg, is_std = calculate_inception_score(images_reals)
print('score', is_avg, is_std)

(50000, 32, 32, 3)
loaded (1000, 32, 32, 3)
score 9.220797 0.7605544


In [ ]:
images_reals.min(), images_reals.max()

(-1.0, 1.0)

In [ ]:
# load cifar10 images
noise = np.random.normal(0, 1, (1000, 100))
# Generate a batch of new images
images = generator.predict(noise)
shuffle(images)
print('loaded', images.shape)
# calculate inception score
is_avg, is_std = calculate_inception_score(images)
print('score', is_avg, is_std)

loaded (1000, 32, 32, 3)
score 6.000495 0.4188693


In [ ]:
images.min(), images.max()

(-0.9991287, 0.9999919)

In [ ]:
images_reals = x_train[:1000]

In [ ]:
images_reals.max()

1.0